In [8]:
import numpy as np
from pathlib import Path

import torch
from transformers import AutoFeatureExtractor, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset, Audio
import evaluate

from scripts.tools import load_pickle

import matplotlib.pyplot as plt
from typing import Tuple, List, Dict
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DATA_PATH = Path('../data/')
random_seed = 54

In [3]:
target_dict = load_pickle(DATA_PATH / "humor_label_sdk.pkl")
audio = [f'../data/urfunny2_audio/{str(idx)}.mp3' for idx in target_dict.keys()]
target_dict = load_pickle(DATA_PATH / "humor_label_sdk.pkl")
target = list(target_dict.values())

audio_ds = Dataset.from_dict({'audio' : audio, 'label': target}).cast_column("audio", Audio(sampling_rate=16_000))

In [4]:
dataset = hug_dataset.train_test_split(test_size=0.3, seed=random_seed)

In [5]:
audio_ds['test'][0]

{'audio': {'path': '../data/urfunny2_audio/14288.mp3',
  'array': array([-6.39972555e-08, -1.94223020e-08,  1.30457778e-07, ...,
          2.84917094e-02,  7.88234547e-03, -6.64830022e-03]),
  'sampling_rate': 16000},
 'label': 1}

In [9]:
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

def preprocess_function(wave):
    audio_arrays = [x["array"] for x in wave["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=16000,
        truncation=True,
    )
    return inputs

C:\Codes\transformers\src\transformers\configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [10]:
encoded_audio_ds = audio_ds.map(preprocess_function, batched=True)

Map:   0%|          | 0/7116 [00:00<?, ? examples/s]

In [ ]:
encoded_audio_ds['train'][0]

{'audio': {'path': '../data/urfunny2_audio/12.mp3',
  'array': array([ 0.00176248,  0.00271043,  0.00079096, ..., -0.10054898,
         -0.0939133 , -0.09656449]),
  'sampling_rate': 16000},
 'label': 0}

In [10]:
label2id = {"HUMOR": 1, "NONHUMOR": 0}
id2label = {0: "NONHUMOR", 1: "HUMOR"}

In [11]:
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [12]:
num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base",
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

/home/timmiakov/miniconda3/envs/audio/lib/python3.10/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['quantizer.weight_proj.weight', 'project_hid.weight', 'project_q.bias', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.bias', 'quantizer.codevectors']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initia

In [13]:
training_args = TrainingArguments(
    output_dir="my_awesome_mind_model",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=1,
    num_train_epochs=2,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_audio_ds["train"],
    eval_dataset=encoded_audio_ds["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/timmiakov/miniconda3/envs/audio/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


TrainOutput(global_step=4, training_loss=0.70558100938797, metrics={'train_runtime': 22.196, 'train_samples_per_second': 0.991, 'train_steps_per_second': 0.18, 'total_flos': 145258122240000.0, 'train_loss': 0.70558100938797, 'epoch': 1.45})

In [14]:
feature_extractor = AutoFeatureExtractor.from_pretrained("../new_features/my_awesome_mind_model/checkpoint-4/")
inputs = feature_extractor(encoded_audio_ds['test'][0]["audio"]["array"], sampling_rate=16000, return_tensors="pt")

In [15]:
model = AutoModelForAudioClassification.from_pretrained("../new_features/my_awesome_mind_model/checkpoint-4/")
model.config.output_hidden_states=True
with torch.no_grad():
    logits = model(**inputs).hidden_states